Goal: *Generate two separate animations of cumulative slip and geometric moment.*

What this notebook does: *Loads data and produces an animation for a single, user-chosen variable as an MP4.*

In [ ]:
import numpy as np
import pyvista as pv
from tqdm import tqdm

from kale import Engine, save_movie, show_ui
from kale.algorithms import (
    contour_banded,
    subdivide_algorithm,
    scalars_operation_algorithm,
)
from kale import theme, helpers

In [ ]:
# Theming adjustments
# pv.global_theme.image_scale = 10

# Load Data

In [ ]:
MESH_GEOMETRY_FILE_NAME = "2022_11_30_11_52_49_mesh_geometry.vtk"
MESH_VALUES_FILE_NAME = "2022_11_30_11_52_49.hdf"
MESH_Z_SCALE = 0.01

In [ ]:
engine = Engine(MESH_GEOMETRY_FILE_NAME, MESH_VALUES_FILE_NAME, zscale=MESH_Z_SCALE)
engine.keys

# Find Camera Position

This is a helper section to find your preferred camera location

In [ ]:
pl = pv.Plotter()
pl.add_mesh(engine.mesh, color="beige")
pl.show()

In [ ]:
# copy value of this output into next cell after moving scene above
pl.camera_position

In [ ]:
# set the camera position (different for different datasets)
cpos = [
    (246.72654935831832, 56.640050977986135, 19.47264855846906),
    (235.64372937685064, 45.67443514953468, -0.9481135945038834),
    (-0.6591979033815945, -0.45237409343002455, 0.6006794517633125),
]

# Choosing data array

The following cell is where you can control which variable is being plotted:

In [ ]:
engine.mesh.active_scalars_name = "geometric_moment"

# Visualize

This cell will not produce an output, but instead renders off-screen to create screenshots.

In [ ]:
# Set contour levels
# CONTOUR_LEVELS = np.linspace(-10e9, 10e9, 11)
CONTOUR_LEVELS = np.linspace(-2500, 2500, 11)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]

contour, edges = contour_banded(
    engine.algorithm,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars=engine.mesh.active_scalars_name,
)

In [ ]:
#### Plotting code ####
pl = pv.Plotter()
pl.add_mesh(engine.boundary)
pl.add_mesh(
    contour,
    cmap=theme.COLOR_MAPS[engine.mesh.active_scalars_name],
    clim=CLIM,
    scalars=engine.mesh.active_scalars_name,
    n_colors=N_COLORS,
    show_scalar_bar=True,
    scalar_bar_args=dict(**theme.SCALAR_BAR_OPTS),
)
pl.add_mesh(edges)

# box = pv.Box(engine.mesh.bounds)
# pl.add_mesh(box, color="lightgrey", culling="front")
pl.add_floor("-z", show_edges=True, edge_color="white", color="lightgray")
helpers.add_bounds(pl)


# Use camera position set earlier
pl.camera_position = cpos
# pl.camera.zoom(1.2)

# Drawing lines as a grid proxy
# mesh = pv.Line((0, 0, 0), (0, 0, 1))
# mesh.plot(color='k', line_width=10)

# Experimenting with lights
light = pv.Light((-2, 2, 0), (0, 0, 0), "white")
pl.add_light(light)


# Make a video over time steps
step = 500
pl.open_movie(f"{engine.mesh.active_scalars_name}.mp4", framerate=60, quality=5)
for tindex in tqdm(range(0, engine.max_time_step, step)):
    engine.time_step = tindex
    pl.add_text(
        f"t = {tindex:05d}", name="time-step-label", font_size=theme.TIMESTEP_FONT_SIZE
    )
    pl.write_frame()
pl.close()

In [ ]:
f"{engine.mesh.active_scalars_name}.mp4"